# Step 1 Build lambda layers for following external packages

In [26]:
import pandas as pd
#from bs4 import BeautifulSoup
#import re
import requests
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [43]:
import sidetable as stb

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
sid = SentimentIntensityAnalyzer()


In [192]:
def get_reviews_data(asin_number):
# set up the request parameters
    params = {
      'api_key': '',
      'type': 'reviews',
      'amazon_domain': 'amazon.com',
      'asin': f'{asin_number}'
    } 

    # make the http GET request to Rainforest API
    api_result = requests.get('https://api.rainforestapi.com/request', params)

    # print the JSON response from Rainforest API
    #df = pd.json_normalize(api_result.json())
    return api_result.json()#df

def get_product_data(asin_number):
# set up the request parameters
    params = {
      'api_key': '',
      'type': 'product',
      'amazon_domain': 'amazon.com',
      'asin': f'{asin_number}'
    } 

    # make the http GET request to Rainforest API
    api_result = requests.get('https://api.rainforestapi.com/request', params)

    # print the JSON response from Rainforest API
   # df = pd.json_normalize(api_result.json())
    return api_result.json()

def get_median_compound(series):
    for asin in df:
        single_review = pd.json_normalize(series)
        single_review['scores'] = single_review.body.apply(lambda review: sid.polarity_scores(review))
        single_review['compound'] = single_review['scores'].apply(lambda d:d['compound'])
        return single_review['compound'].median()
    
def get_median_neg(series):
    for asin in df:
        single_review = pd.json_normalize(series)
        single_review['scores'] = single_review.body.apply(lambda review: sid.polarity_scores(review))
        single_review['neg'] = single_review['scores'].apply(lambda d:d['neg'])
        return single_review['neg'].median()
    
def get_median_neu(series):
    for asin in df:
        single_review = pd.json_normalize(series)
        single_review['scores'] = single_review.body.apply(lambda review: sid.polarity_scores(review))
        single_review['neu'] = single_review['scores'].apply(lambda d:d['neu'])
        return single_review['neu'].median()

def get_median_pos(series):
    for asin in df:
        single_review = pd.json_normalize(series)
        single_review['scores'] = single_review.body.apply(lambda review: sid.polarity_scores(review))
        single_review['pos'] = single_review['scores'].apply(lambda d:d['pos'])
        return single_review['pos'].median()

# RainForest Api to get Product Data

In [12]:
Asin = ['B07TWNMPWC', 'B0012JY42I', 'B001E8ISJW', 'B004LL7AXE', 'B07H8P1DRD', 'B08HJBTZMH', 
'B00IEC63ZM', 'B01C8BR7XG', 'B074VG5B46', 'B0891Z56YH', 'B075RFJLSG', 'B00TUCFBI0',
'B07RC8CXC1', 'B07RCHRS9N', 'B098JYL6FV', 'B07PHVLPL9', 'B08DKQV6H1', 'B07H8P1YDJ', 
'B00IXRZWW8', 'B00CM8L32M', 'B01C8BR84O', 'B001Y8EOHW', 'B07TVKQLWN', 'B00EEEITVA',
'B08S39XVGC', 'B08P69MZQP', 'B00854L57U', 'B00KAP0HQW', 'B000UEBH8A', 'B072F8HRM1',
'B08WW9QR7H', 'B0091CG65G', 'B07PFLKLLR', 'B075RDZ3WF', 'B07N7ZM39Z', 'B07TXRKXPR',
'B00U10LW74', 'B07R5QF3S7', 'B0018SGX6I', 'B01ANSJV80', 'B016KKEWHA', 'B00Z2LOKS0', 
'B01C8BR83A', 'B0025ASIRK', 'B074BLLV81', 'B07N813336', 'B079G4N8K2', 'B007G7JP2O', 
'B004RR1FF6', 'B08Y66H6B7']

In [20]:
asins = pd.DataFrame(Asin, columns=['Asin']) 

In [193]:
%%time

ihealthproductdf = pd.DataFrame()

for asin in asins.Asin:
    asin_single_product = get_product_data(asin)
    ihealthproductdf = ihealthproductdf.append(pd.json_normalize(asin_single_product), ignore_index=True)

Wall time: 5min 55s


In [226]:
import numpy as np
ihealthproductdf['is_percentage'] = np.where(ihealthproductdf['product.coupon_text'].str.contains('%', na=False, regex=False), 1, 0)
ihealthproductdf['is_dollar'] = np.where(ihealthproductdf['product.coupon_text'].str.contains('$', na=False, regex=False), 1, 0)

# RainForest Api to get Amazon Reviews Data

In [178]:
%%time

ihealthreviewdf = pd.DataFrame()

for asin in asins.Asin:
    asin_single_review = get_reviews_data(asin)
    ihealthreviewdf = ihealthreviewdf.append(pd.json_normalize(asin_single_review), ignore_index=True)

Wall time: 3min 34s


# Adding median sentiment analysis scores for front page reviews for each Asin 

In [320]:
ihealthreviewdf['reviews_sentiment_compound_score'] = backup.reviews.apply(lambda review: get_median_compound(review))
ihealthreviewdf['reviews_sentiment_neg_score'] = backup.reviews.apply(lambda review: get_median_neg(review))
ihealthreviewdf['reviews_sentiment_pos_score'] = backup.reviews.apply(lambda review: get_median_pos(review))
ihealthreviewdf['reviews_sentiment_neu_score'] = backup.reviews.apply(lambda review: get_median_neu(review))


In [323]:
ihealthreviewdf.head(3).T

,0,1,2
request_parameters.asin,B07TWNMPWC,B0012JY42I,B001E8ISJW
reviews,"[{'id': 'R35DCHT9WTU2TI', 'title': 'So disappo...","[{'id': 'R1MM9MRG6ORDJA', 'title': 'Exactly as...","[{'id': 'R1SNOEMJ5PHRRP', 'title': 'Worth it.'..."
request_info.success,True,True,True
request_info.credits_used,6,7,8
request_info.credits_used_this_request,1,1,1
request_info.credits_remaining,494,493,492
request_info.credits_reset_at,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z
request_metadata.created_at,2021-11-03T04:57:50.770Z,2021-11-03T04:57:53.527Z,2021-11-03T04:57:58.619Z
request_metadata.processed_at,2021-11-03T04:57:53.074Z,2021-11-03T04:57:57.689Z,2021-11-03T04:58:01.004Z
request_metadata.total_time_taken,2.3,4.16,2.38


# Getting granular look at the front page reviews for Asin: B07TWNMPWC

In [331]:
pd.json_normalize(ihealthreviewdf.reviews[0]).head(3).T

,0,1,2
id,R35DCHT9WTU2TI,R31KPH44OYPTTW,RMSF3SYP32PIY
title,So disappointing..,Yay for regularity!,Can’t get in touch with manufacturer.
body,I don’t know what happened to these gummies......,I’ve had digestive issues forever and I also h...,Loved this probiotic but my last bottle had a ...
body_html,<span> I don’t know what happened to these gum...,<span> I’ve had digestive issues forever and I...,<span> Loved this probiotic but my last bottle...
link,https://www.amazon.com/gp/customer-reviews/R35...,https://www.amazon.com/gp/customer-reviews/R31...,https://www.amazon.com/gp/customer-reviews/RMS...
rating,2,5,5
vine_program,False,False,False
verified_purchase,True,True,True
helpful_votes,10.0,13.0,13.0
review_country,us,us,us


In [32]:
productcolumns = ihealthproductdf.columns.to_list()

In [42]:
ihealthproductdf.head(5).T

,0,1,2,3,4
also_viewed,[{'title': 'Culturelle Daily Probiotic Gummies...,NaN,NaN,NaN,NaN
compare_with_similar,"[{'asin': 'B07TWNMPWC', 'image': 'https://m.me...","[{'asin': 'B0012JY42I', 'image': 'https://m.me...","[{'asin': 'B001E8ISJW', 'image': 'https://m.me...","[{'asin': 'B004LL7AXE', 'image': 'https://m.me...","[{'asin': 'B07H8P1DRD', 'image': 'https://m.me..."
sponsored_products,[{'title': 'Culturelle Daily Probiotic Gummies...,[{'title': 'AZO Dual Protection | Urinary + Va...,[{'title': 'Vagistat 1 Day Single-Dose Yeast I...,[{'title': 'Freshfield Vegan Omega 3 DHA Suppl...,[{'title': 'up4 Probiotic Supplement for Women...
request_info.success,True,True,True,True,True
request_info.credits_used,56,57,58,59,60
request_info.credits_used_this_request,1,1,1,1,1
request_info.credits_remaining,444,443,442,441,440
request_info.credits_reset_at,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z,2021-12-03T04:57:06.000Z
request_metadata.created_at,2021-11-03T05:11:19.983Z,2021-11-03T05:11:27.055Z,2021-11-03T05:11:32.853Z,2021-11-03T05:11:37.861Z,2021-11-03T05:11:43.709Z
request_metadata.processed_at,2021-11-03T05:11:26.387Z,2021-11-03T05:11:32.251Z,2021-11-03T05:11:37.258Z,2021-11-03T05:11:43.114Z,2021-11-03T05:11:49.068Z


# Product API Missing Frequency Report

In [44]:
ihealthproductdf.stb.missing(style=True)

,missing,total,percent
product.protection_plans,48,50,96.00%
product.amazons_choice.link,48,50,96.00%
product.documents,48,50,96.00%
product.recommended_age,48,50,96.00%
product.buybox_winner.availability.stock_level,46,50,92.00%
product.buybox_winner.fulfillment.third_party_seller.link,45,50,90.00%
product.buybox_winner.fulfillment.third_party_seller.name,45,50,90.00%
product.buybox_winner.fulfillment.third_party_seller.id,45,50,90.00%
product.coupon_text,44,50,88.00%
product.has_coupon,44,50,88.00%


# Review API Missing Report

In [48]:
ihealthreviewdf.stb.missing(style=True)

,missing,total,percent
pagination.message,43,50,86.00%
product.attributes,19,50,38.00%
top_critical.helpful_votes,1,50,2.00%
top_positive.verified_purchase,0,50,0.00%
top_critical.body,0,50,0.00%
top_critical.title,0,50,0.00%
top_critical.id,0,50,0.00%
top_positive.is_global_review,0,50,0.00%
top_positive.review_country,0,50,0.00%
top_positive.helpful_votes,0,50,0.00%
